# 评估分类模型

分类模型相对是比较好评估的,一般来说我们在训练时会将训练数据分为`训练集,验证集和测试集`三份或者`训练集,验证集`两份当模型训练出来的时候我们可以拿测试集或者没有的话用验证集来作为评估的目标.先定义几个参数:

+ 预测为A且实际为A的 叫做 tp
+ 预测为A且实际为!A的 叫做 fp
+ 预测为!A且实际为A的 叫做 fn
+ 预测为!A且实际为!A的 叫做tn

### 准确度(accuracy)

指在一定实验条件下多次测定的平均值与真值相符合的程度,以误差来表示.它用来表示系统误差的大小.其分子为预测对的数量,分母为全部预测数据数量

$$ accuracy=\frac {tp+tn}{tp+fp+tn+fn} $$

### 精确率(precision)

$$ precision = \frac {tp}{tp+fp}$$

准确率描述预测为A的准确情况

### 召回率(recall)


也叫敏感性指标(sensitivity)

$$ recall = \frac {tp}{tp+fn} $$

召回率描述实际是A的被预测准确率

### 特异性指标(Specificity)

$$ Specificity=\frac{tn}{fp+tn} $$

### 负正类率(false positive rate, FPR)

$$ FPR=\frac {fp}{fp+tn}$$

### f1_score

精确率和召回率都不能完全描述一个模型的准确程度.f1则是用于组合这两个数据以获取一个评价标准/


$$ f1 = 2 * \frac {precision * recall}{precision + recall} $$

### 混淆矩阵(confusion matrix)

混淆矩阵是更加细致的一个模型效果评估工具,它的纵轴为真实标签,横轴为预测标签,其中的数字则是命中的个数.通过这个可以比较清洗的看出模型预测的偏向性,以此可以作为模型调整的依据.